In [ ]:
from utils.db import get_connection
from dotenv import load_dotenv
import utils.links as links
from pathlib import Path
from time import sleep
import pandas as pd
import requests
import uuid
import json
import os

load_dotenv()

try:
    api_key = os.getenv("API_KEY") 

    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }

    conn = get_connection()
    cursor = conn.cursor()
    print("Connection Successful")
except Exception as e:
    print("Connection Failed: ", e)

In [ ]:
def safe_json(url):
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        raise ValueError(f"Request failed {r.status_code}: {r.text[:200]}")
    try:
        return r.json()
    except Exception:
        raise ValueError(f"Invalid JSON response from {url}: {r.text[:200]}")

<h1>Seasons Table</h1>

In [ ]:
# Insertion of seasons table
try:
    season_res = requests.get(links.seasons_url,headers=headers)
    season_res = season_res.json()

    insert_query = "INSERT INTO seasons (season_id, year, start_date, end_date, status, type_code) VALUES  (%s, %s, %s, %s, %s, %s);"

    # Combining rows into a list
    season_rows = [
        (row["id"],row["year"],row["start_date"],row["end_date"],row["status"],row["type"]["code"])
        for row in season_res["seasons"]
    ]
    
    cursor.executemany(insert_query,season_rows)
    conn.commit()

    print("Insertion in Season Table Successful")
except Exception as e:
    print("Error Occurred: ", e)

<h1>Venues Table</h1>

In [ ]:
# Insertion of venues table
try:
    # Fetching season years for API parameter
    cursor.execute("SELECT year FROM seasons ORDER BY year DESC;")
    season_years = [r[0] for r in cursor.fetchall()]
    
    # Insert Query
    insert_query = "INSERT IGNORE INTO venues (venue_id, name, city, state, country, zip, address, capacity, surface, roof_type, latitude, longitude) VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

    for season_year in season_years:
        venues_res = requests.get(links.venues_url.format(season_year=season_year),headers=headers)
        venues_res = venues_res.json()
        for week in venues_res["weeks"]:
            for game in week["games"]:
                venue = game.get("venue", {})
                if not venue:
                    continue

                id = venue.get("id")
                name = venue.get("name")
                city = venue.get("city")
                state = venue.get("state")
                country = venue.get("country")
                zip = venue.get("zip")
                address = venue.get("address")
                capacity = venue.get("capacity")
                surface = venue.get("surface")
                roof_type = venue.get("roof_type")
                latitude = venue.get("location", {}).get("lat")
                longitude = venue.get("location", {}).get("lng")

                if not all([id,name,city,state,country,zip,address,capacity,surface,roof_type,latitude,longitude]):
                    continue

                cursor.execute(insert_query, (id,name,city,state,country,zip,address,capacity,surface,roof_type,latitude,longitude))

    conn.commit()

    print("Insertion in Venue Table Successful")
except Exception as e:
    print("Error Occurred: ", e)

<h1>Divisions & Conferences Table</h1>

In [ ]:
# Insertion of table divisions and conferences
try:
    # Fetching season years for API parameter
    cursor.execute("SELECT year FROM seasons ORDER BY year DESC;")
    season_years = [r[0] for r in cursor.fetchall()]

    divisions_insert_query = "INSERT IGNORE INTO divisions (division_id, name, alias) VALUES (%s, %s, %s);" 
    conferences_insert_query = "INSERT IGNORE INTO conferences (conference_id, name, alias) VALUES (%s, %s, %s);" 

    divisions_rows = []
    conferences_rows = []

    for season_year in season_years:
        postgame_res = requests.get(links.divisions_url.format(season_year=season_year),headers=headers).json()
        for div in postgame_res["divisions"]:
            divisions_rows.append((div["id"], div["name"], div["alias"]))
            for conf in div["conferences"]: 
                conferences_rows.append((conf["id"], conf["name"], conf["alias"]))

    cursor.executemany(divisions_insert_query,divisions_rows)
    cursor.executemany(conferences_insert_query,conferences_rows)
    conn.commit()

except Exception as e:
    print("Error Occurred: ", e)


<h1>Teams, Players & Coaches Table</h1>

In [ ]:
# Saving all teams roaster data locally because of API rate limits
try:
    OUTPUT_DIR = "./data/team_rosters/"

    print("Requesting for team data")
    teams_data = safe_json(links.teams_url)
    all_teams = teams_data.get("teams", [])
    print("Received Team Data")
    
    team_no = 0
    first_batch_teams = all_teams[team_no:]

    print("Fetching for team roaster data")
    for team in first_batch_teams:
        sleep(1) # Query Per Second : 1 second
        team_id = team.get("id")
        print(f"Request for team Idx: {team_no}")
        url = links.players_url.format(team_id=team_id)
        
        roaster = safe_json(url)
        team_id = roaster.get("id")

        filename = f"team_{team_id}.json"
        filepath = os.path.join(OUTPUT_DIR, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(roaster, f, indent=4, ensure_ascii=False)
        
        print(f"File written- team Idx: {team_no}")
        team_no += 1

except Exception as e:
    print("Error:", e)

In [ ]:
# Data cleaning functions for table teams, players and coaches
def get_team_data(data): 
    team_id = data.get("id")
    market = data.get("market")
    name = data.get("name")
    alias = data.get("alias")
    founded = data.get("founded")
    mascot = data.get("mascot")
    fight_song = data.get("fight_song")
    championships_won = data.get("championships_won")
    conference_id = data.get("conference", {}).get("id")
    division_id = data.get("division", {}).get("id")
    venue_id = data.get("venue", {}).get("id")

    if not all((team_id, market, name, alias, founded, mascot, fight_song, conference_id, division_id, venue_id)):
        return None

    return (team_id, market, name, alias, founded, mascot, fight_song, championships_won, conference_id, division_id, venue_id)


def get_player_data(data, team_id): 
    player_id = data.get("id")
    first_name = data.get("first_name")
    last_name = data.get("last_name")
    abbr_name = data.get("abbr_name")
    birth_place = data.get("birth_place")
    position = data.get("position")
    height = data.get("height")
    weight = data.get("weight")
    status = data.get("status")
    eligibility = data.get("eligibility")

    player_data = (player_id, first_name, last_name, abbr_name, birth_place, position, height, weight, status, eligibility, team_id)

    if not all(player_data): 
        return None
    
    return player_data


def get_coach_data(data, team_id): 
    coach_id = data.get("id")
    full_name = data.get("full_name")
    position = data.get("position")

    if not all((coach_id, full_name, position, team_id)): return None

    return (coach_id, full_name, position, team_id)

In [ ]:
root = Path("./data/team_rosters/")

teams_row = []
coaches_row = []
players_row = []

for file in root.glob("team_*.json"):
    with open(file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            team_data = get_team_data(data)
            if team_data: 
                teams_row.append(team_data)

                # Coaches Table Data Cleaning
                for coach in data["coaches"]: 
                    coach_data = get_coach_data(coach,data["id"])
                    if coach_data: coaches_row.append(coach_data)

                # Players Table Data Cleaning
                for player in data["players"]: 
                    player_data = get_player_data(player,data["id"])
                    if player_data: players_row.append(player_data)

        except Exception as e:
            print(f"Error parsing {file}: {e}")

try:
    # Adding Teams, Players and Coaches Data to DB
    team_insert_query = "INSERT IGNORE INTO teams(team_id, market, name, alias, founded, mascot, fight_song, championships_won, conference_id, division_id, venue_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
    coach_insert_query = "INSERT IGNORE INTO coaches(coach_id, full_name, position, team_id) VALUES(%s, %s, %s, %s)"
    player_insert_quey = "INSERT IGNORE INTO players(player_id, first_name, last_name, abbr_name, birth_place, position, height, weight, status, eligibility, team_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    cursor.executemany(team_insert_query, teams_row)
    cursor.executemany(coach_insert_query, coaches_row)
    cursor.executemany(player_insert_quey, players_row)

    conn.commit()
    print("Data Insertion in teams, players and coaches successful")
except Exception as e:
    print("Error while inserting data: ", e)

<h1>Players Statistics Table</h1>

In [ ]:
# Data cleaning fn for player_statistics tables
def get_stats_data(stat_data, player_data, team_id):
    stat_id = str(uuid.uuid4())
    player_id = player_data.get("id")
    season_id = stat_data.get("season", {}).get("id")

    if not all((stat_id, player_id, season_id)): 
        return None
    
    return (
        stat_id,
        player_data.get("games_played", 0),
        player_data.get("games_started", 0),
        player_data.get("rushing", {}).get("yards", 0),
        player_data.get("rushing", {}).get("touchdowns", 0),
        player_data.get("receiving", {}).get("yards", 0),
        player_data.get("receiving", {}).get("touchdowns", 0),
        player_data.get("kick_returns", {}).get("yards", 0),
        player_data.get("fumbles", {}).get("fumbles", 0),
        player_id,
        team_id,
        season_id
    )

In [ ]:
try:
    # Fetching season years & team_ids for API parameter
    cursor.execute("SELECT year FROM seasons ORDER BY year DESC;")
    season_years = [r[0] for r in cursor.fetchall()]
    cursor.execute("SELECT team_id FROM teams;")
    team_ids = [r[0] for r in cursor.fetchall()]

    insert_query = "INSERT IGNORE INTO player_statistics (stat_id, games_played, games_started, rushing_yards, rushing_touchdowns, receiving_yards, receiving_touchdowns, kick_return_yards, fumbles, player_id, team_id, season_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    batch_size = 1000

    for season_year in season_years:
        player_stat_rows = []
        print(f"Season Start: {season_year}")
        idx = 0
        for team_id in team_ids:
            print(f"Team Start: {idx}")
            sleep(1.5)
            url = links.player_statistics_url.format(season_year=season_year, team_id=team_id)
            seasonal_stats_data = safe_json(url)

            if seasonal_stats_data.get("players", []):
                for player in seasonal_stats_data.get("players"):
                    player_stat = get_stats_data(seasonal_stats_data, player, team_id)
                    player_stat_rows.append(player_stat)
            print(f"team done: {idx}")
            idx += 1
        print(f"team loop: {idx}")
        print(f"Done Year {season_year}")

    for i in range(0, len(player_stat_rows), batch_size):
        batch = player_stat_rows[i : i + batch_size]
        cursor.executemany(insert_query, batch)
        conn.commit()
except Exception as e:
    print("Error Occurred: ", e)

<h1>Rankings Table</h1>

In [ ]:
# Data cleaning fn for rankings tables
def team_exists(team_id):
    cursor.execute("SELECT 1 FROM teams WHERE team_id = %s LIMIT 1", (team_id,))
    return cursor.fetchone() is not None

def get_ranking_data(data, team_data, season_id):
    team_id = team_data.get("id")
    if not team_exists(team_id): return None

    poll_id = data.get("poll", {}).get("id")
    poll_name = data.get("poll", {}).get("name")
    week = data.get("week")
    effective_time = data.get("effective_time")

    if not all((poll_id, poll_name, week, effective_time, team_id)):
        return None
    
    return (
        poll_id,
        poll_name,
        week,
        effective_time,
        team_data.get("rank", 0),
        team_data.get("prev_rank", 0),
        team_data.get("points", 0),
        team_data.get("fp_votes", 0),
        team_data.get("wins", 0),
        team_data.get("losses", 0),
        team_data.get("ties", 0),
        team_id,
        season_id
    )

In [ ]:
cursor.execute("SELECT season_id, year FROM seasons ORDER BY year DESC;")
seasons = [(r[0], r[1]) for r in cursor.fetchall()]

In [ ]:
try:
    # Fetching season years for API parameter & ids for reference key
    cursor.execute("SELECT season_id, year FROM seasons ORDER BY year DESC;")
    seasons = [(r[0], r[1]) for r in cursor.fetchall()]

    week_numbers = [n for n in range(1,22)]
    poll_types = ["AP25", "EU25", "CFP25", "FCS25", "FCSC25"]
    batch_size = 1000

    insert_query = "INSERT INTO rankings(poll_id, poll_name, week, effective_time, current_rank, prev_rank, points, fp_votes, wins, losses, ties, team_id, season_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

    ranking_rows = []
    for poll_type in poll_types:
        print(f"Starting for poll :{poll_type}")
        for season_id, year in seasons:
            print(f"Starting for season: {year}")
            for week_num in week_numbers:
                print(f"Starting for week: {week_num}")
                sleep(1.5)
                url = links.rankings_url.format(poll_type=poll_type,season_year=year,week_number=week_num)
                response = safe_json(url)

                if response.get("rankings", []):
                    for team in response.get("rankings"):
                        ranking_data = get_ranking_data(response, team, season_id)
                        if ranking_data: ranking_rows.append(ranking_data)

                print(f"Done with week: {week_num}")
            print(f"Done with season: {year}")
        print(f"Done with poll: {poll_type}")

    for i in range(0, len(ranking_rows), batch_size):
        batch = ranking_rows[i : i + batch_size]
        cursor.executemany(insert_query, batch)
        conn.commit()

    print("Insertion in table rankings successful")
except Exception as e:
    print("Error occurred: ", e)


In [ ]:
cursor.close()
conn.close()